# CUT: Semiotics of the Kitchen

This colab notebook is adapated from the Derrick Schultz's notebook. The goal of this project is to regenerate an input video frame by frame in a new style using the CUT model.


Original Colab by Derrick Schultz:

[Notebook](https://colab.research.google.com/github/dvschultz/Make-ML-Art-with-Google-Colab/blob/master/CUT_inference.ipynb)

CUT sources:

[GitHub](https://github.com/taesungp/contrastive-unpaired-translation) | [Website](http://taesung.me/ContrastiveUnpairedTranslation/)

##Set-up

Set up Colab

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Downloading the CUT model

In [ ]:
# navigate to the directory in which to download the model
%cd /content/drive/MyDrive/CCI_modular/TERM_03/

/content/drive/MyDrive/CCI_modular/TERM_03


In [ ]:
# clone the CUT github repo
!git clone https://github.com/taesungp/contrastive-unpaired-translation CUT

In [ ]:
# install libraries
!pip install dominate visdom GPUtil

## Import custom dataset

The dataset is a zipped folder in Google Drive with images frames of Rosler's "Semiotics of the Kitchen" and scrapped YouTube videos. The files are organized into training and test sets (trainA, trainB, testA, and testB).

In [ ]:
# unzip the custom dataset into ./datasets
# NOTE: this takes awhile!
%cd /content/drive/MyDrive/CCI_modular/TERM_03/
!unzip kitchen_02.zip -d /content/drive/MyDrive/CCI_modular/TERM_03/CUT/datasets

/content/drive/MyDrive/CCI_modular/TERM_03
Archive:  kitchen_02.zip


## Train the model


In [ ]:
# navigate back to the CUT folder
%cd /content/drive/MyDrive/CCI_modular/TERM_03/CUT

/content/drive/MyDrive/CCI_modular/TERM_03/CUT


In [ ]:
# set up training parameters
dataset_path = './datasets/kitchen_02'
project_name = 'kitchen_02'
load_size = 256 # input images are 256*256
crop_size = 256

In [ ]:
# initial training
!python train.py --dataroot $dataset_path --name $project_name --CUT_mode CUT --load_size $load_size --crop_size $crop_size --display_id 0

Run the cell below to continue training the model from the last saved epoch.

*   `--continue_train`: continue training the model

*Note that the epochs start from 1 instead of the count of the last epoch from the previous training session.*

In [ ]:
#continue training
!python train.py --dataroot $dataset_path --name $project_name --CUT_mode CUT --load_size $load_size --crop_size $crop_size --display_id 0 --continue_train

Let's have a look at the checkpoints

In [ ]:
# list the checkpoints
!ls ./checkpoints/kitchen_02/

10_net_D.pth  15_net_G.pth  25_net_F.pth  latest_net_D.pth  train_opt.txt
10_net_F.pth  20_net_D.pth  25_net_G.pth  latest_net_F.pth  web
10_net_G.pth  20_net_F.pth  5_net_D.pth   latest_net_G.pth
15_net_D.pth  20_net_G.pth  5_net_F.pth   loss_log.txt
15_net_F.pth  25_net_D.pth  5_net_G.pth   test_opt.txt


In [ ]:
# from google.colab import files
# files.download('./checkpoints/kitchen_CUT/latest_net_F.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Test the model



*   `--num_test`: how many test images to produce
*   `--direction`: `AtoB` or `BtoA`

In [ ]:
!python test.py --dataroot ./datasets/kitchen/ --name kitchen_CUT --CUT_mode CUT --num_test 50 --direction AtoB

##Regenerating the input video

I created a dataset with all 4000+ frames from the input video as "testA" and all the 2000+ frames from scrapped YouTube videos as "testB". Although not used during image generation, I also had to include "trainA" and "trainB" folders with 5 images each.

Unzip folders with video frames to create dataset

In [ ]:
# get to the directory where the .zip folders are stored
%cd /content/drive/MyDrive/CCI_modular/TERM_03/kitchen_frames/

# unzip input video frames & download into specific directory
# the file be renamed to "testA"
!unzip kitchen_testA.zip -d /content/drive/MyDrive/CCI_modular/TERM_03/kitchen_frames/kitchen_v3/

# unzip YouTube video frames & download into specific directory
# the file will be renamed to "testB"
# !unzip youtube_testB.zip -d /content/drive/MyDrive/CCI_modular/TERM_03/kitchen_frames/kitchen_v3/

In [ ]:
# CREDIT: the following code was referenced from chatGPT
import os

# define input and output directories
input_dir = '/content/drive/MyDrive/CCI_modular/TERM_03/kitchen_frames/kitchen_v3'
output_dir = '/content/drive/MyDrive/CCI_modular/TERM_03/kitchen_frames/kitchen_v3'

# create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# get a sorted list of files in the input directory
files = sorted(os.listdir(input_dir))

Perform some checks before generating frames

In [ ]:
# check the that testA looks ok
!ls /content/drive/MyDrive/CCI_modular/TERM_03/kitchen_frames/kitchen_v3/testA

In [ ]:
# check the that testB looks ok
!ls /content/drive/MyDrive/CCI_modular/TERM_03/kitchen_frames/kitchen_v3/testB

Generate frames by calling *test.py*

*   `--data_root`: set as `input_dir`
*   `--results_dir`: set as `output_dir`
*   `--num_test`: set to 4417, the number of files in testA
*   `--serial_batches`: `True`, intake the files in order

In [ ]:
# cd back to the CUT folder
%cd /content/drive/MyDrive/CCI_modular/TERM_03/CUT

# generate images
!python test.py --dataroot {input_dir} --results_dir {output_dir} --name kitchen_02 --CUT_mode CUT --num_test 4417 --direction AtoB --serial_batches

Download the generated frames

In [ ]:
# zip the file
!zip -r '/content/drive/MyDrive/CCI_modular/TERM_03/kitchen_frames/kitchen_v3/kitchen_02/test_latest/images/fake_B.zip' '/content/drive/MyDrive/CCI_modular/TERM_03/kitchen_frames/kitchen_v3/kitchen_02/test_latest/images/fake_B'

In [ ]:
from google.colab import files

# download the file
files.download('/content/drive/MyDrive/CCI_modular/TERM_03/kitchen_frames/kitchen_v3/kitchen_02/test_latest/images/fake_B.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>